<h1> FOOFing OMEGA Data </h1>

This notebook runs FOOF on MEG-PSD data.
- Option to Parallelize with Pool from Multiprocessing
- Must Specify frequency range desired for FOOF
- Saves FOOF results in pickled object

NOTES:
- For reasons unknown, parallel implementation does not currently work. 

    -It used to, but now hangs when trying to parallelize foof. 

In [2]:
# Import sys and add FOOF location to path
import sys
sys.path.append('/Users/thomasdonoghue/Documents/GitCode/')

# Import FOOF
from foof import syn
from foof.fit import FOOF

# Import general code from custom module om
sys.path.append('/Users/thomasdonoghue/Documents/GitCode/omegamappin/')
from om.gen import *

# Import Parallelization Packages
from multiprocessing import Pool, freeze_support

# Import other packages
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Set plots inline
%matplotlib inline

In [ ]:
# Set Paths to MEG Data
meg_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/2-PSDs/Colin/'

In [ ]:
# Check Availabe Subjects
files = os.listdir(meg_path)
files = clean_file_list(files, 'Subject_')
print "Number of Subjects", len(files)
print files

In [ ]:
# Load MEG Data
subj = 559176
psd, freqs = load_meg_psds(meg_path, subj)

In [ ]:
# Check data - get number of PSDs and frequency resolution
[nPSDs, nFreqs] = np.shape(psd)
freq_res = np.mean(np.diff(freqs))
print 'Number of PSDs is: ', nPSDs
print 'Frequency Resolution is: ', freq_res

In [ ]:
# Extract PSD range of interest
psd_ext, freqs_ext = extract_psd(psd, freqs, f_low=3, f_high=40)

In [ ]:
# Run FOOF
foof_results = meg_foof(psd_ext, freqs_ext, min_p=0.1, freq_res=freq_res, method='linear')

# Save out results
save_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/Colin/pickle'
save_pickle(foof_results, save_path, subj)

### Convert pickle files (of saved FOOF results) to csv

In [ ]:
## To convert a single file

# Set subject number to convert
subj = 559176

# Set paths to load and save to/from
dat_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/Colin/'
csv_save_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/Colin/'

# Load data from pickle file and resave as csv file
dat = load_pickle(dat_path, subj)
save_csv(dat, csv_save_path, subj)

In [ ]:
import os
files = os.listdir(os.path.join(dat_path, 'pickle'))
cur_subj_file = get_cur_subj(subj, files)
subj_path = os.path.join(dat_path, 'pickle', cur_subj_file)

a = pickle.load(open(subj_path, 'rb'))

In [ ]:
# To convert all files

# Set paths
dat_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/Colin/'
csv_save_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/Colin/'

# Get list of pickle files available
files = os.listdir(meg_path)
files = clean_file_list(files, 'Subject_')

# Get list of subj numbers for all files available
sub_nums = get_sub_nums(files)

# Loop through each subj, load, and save back out as csv
for subj in sub_nums:
    
    # Load subj foof data, from pickle file
    dat = load_pickle(dat_path, subj)
    
    # Save out as csv file
    save_csv(dat, csv_save_path, subj)

In [ ]:
# Open Pool, run FOOF in parallel, and then close pool.  - OLD
# Results stored in list of tuples"
pool = Pool(8)
results = pool.map(foof_par, psd_list[0:1])
pool.close()
pool.join()

# Use pickle to save the results
foof_data_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/'
foof_save_name = str(subj) + '_Foof_Vertex.p'
foof_save_path = os.path.join(foof_data_path, foof_save_name)
pickle.dump(results, open(foof_save_path, 'wb'))

# Close the pool
pool.terminate()

In [ ]:
# Parallel not working. Run linearly.  - OLD
results = [foof_par(psd) for psd in psd_list]

# Use pickle to save the results
foof_data_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/'
foof_save_name = str(subj) + '_Foof_Vertex.p'
foof_save_path = os.path.join(foof_data_path, foof_save_name)
pickle.dump(results, open(foof_save_path, 'wb'))

In [ ]:
# EXTRA: CHECK FOOF

print("model: f {0}, f_sig {1}, chi {2}".format(
        np.round(foof.centers_, decimals=1),
        np.round(foof.stdevs_, decimals=1),
        np.round(foof.chi_, decimals=1))
     )

plt.subplot(211)
plt.plot(freqs_ext, psd_example[:,0]*10000000000000000000000, label='Data')
plt.plot(*syn.mfonef(mk=foof.powers_, 
                     mf=foof.centers_, 
                     mf_sig=foof.stdevs_, 
                     f0=3, 
                     fmax=50, 
                     chi=foof.chi_, 
                     res=freq_res), 
         label='Model'
        )
plt.xlabel("F (Hz)")
plt.ylabel("PSD")
plt.yscale('log')
plt.xscale('log')
plt.legend()